In [219]:
# the goal of this notebook is to scrape the data from https://www.fantasypros.com/mlb/rankings/ros-overall.php

# library load
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import pandas as pd

In [220]:
import requests
from bs4 import BeautifulSoup

url = "https://www.fantasypros.com/mlb/rankings/ros-overall.php?eligibility=E"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    inner_elements = soup.find_all(class_="inner")
    
    for element in inner_elements:
        print(element.get_text())
else:
    print("Failed to retrieve the webpage.")




2024 Fantasy Baseball ROS Rankings
Expert Consensus
                            Ranking (5 of 5 Experts) - Mar 29, 2024


Pick Experts



Options 
 Import a Team Download CSV
 Printer Friendly Version
 Pick Experts



 




OverallHittersPitchers1B2B3BSSCOFSPRPDH 

Settings





Options 
 Import a Team Download CSV
 Printer Friendly Version
 Pick Experts



 



MLB Leagues


Mixed
AL-Only
NL-Only




Position


Overall OverallHittersPitchers1B2B3BSSCOFSPRPDH 

View


Ranks
Notes 




Import a Team

Eligibility


Combined
Yahoo
ESPN
CBS










Rank
                                
Player
                                    (Team, Position)
                                    

Best
Worst
Avg
Std Dev
ADP
vs. ADP
Notes



1
Ronald Acuna Jr. (ATL - RF) 1
1
1.0
0.0
1.0
 ‐
 

2
Bobby Witt Jr. (KC - SS) 2
3
2.2
0.4
4.0
 +2.0
 

3
Julio Rodriguez (SEA - CF) 3
4
3.2
0.4
2.0
 -1.0
 

4
Mookie Betts (LAD - 2B,RF) 2
5
4.0
1.1
3.0
 -1.0
 

5
Corbin Carroll (ARI - LF,CF,RF) 4
7
5.8
1.0
5.0
 ‐

In [221]:
from bs4 import BeautifulSoup

# Assuming inner_elements is a bs4.element.ResultSet object
# Convert inner_elements to text
inner_text = ' '.join([element.get_text().strip() for element in inner_elements])

# Filter out text before the word "Rank"
rank_index = inner_text.find("Rank")
if rank_index != -1:
    inner_text = inner_text[rank_index:]

# Split the text into a list of items
items_list = inner_text.split('\n')

# Remove empty strings from the list
items_list = [item for item in items_list if item.strip()]

# Now, items_list contains the list of items after trimming white spaces and filtering out text before "Rank"
print(items_list)

# remove all items before  'Rank\r'
for i in range(len(items_list)):
    if items_list[i] == 'Rank\r':
        items_list = items_list[i:]
        break

print(items_list)

# trim whitespace
items_list = [item.strip() for item in items_list]


print(items_list)

['Rankings', 'Expert Consensus\r', '                            Ranking (5 of 5 Experts) - Mar 29, 2024', 'Pick Experts', 'Options ', ' Import a Team Download CSV', ' Printer Friendly Version', ' Pick Experts', 'OverallHittersPitchers1B2B3BSSCOFSPRPDH ', 'Settings', 'Options ', ' Import a Team Download CSV', ' Printer Friendly Version', ' Pick Experts', 'MLB Leagues', 'Mixed', 'AL-Only', 'NL-Only', 'Position', 'Overall OverallHittersPitchers1B2B3BSSCOFSPRPDH ', 'View', 'Ranks', 'Notes ', 'Import a Team', 'Eligibility', 'Combined', 'Yahoo', 'ESPN', 'CBS', 'Rank\r', 'Player\r', '                                    (Team, Position)\r', 'Best', 'Worst', 'Avg', 'Std Dev', 'ADP', 'vs. ADP', 'Notes', '1', 'Ronald Acuna Jr. (ATL - RF) 1', '1', '1.0', '0.0', '1.0', ' ‐', '2', 'Bobby Witt Jr. (KC - SS) 2', '3', '2.2', '0.4', '4.0', ' +2.0', '3', 'Julio Rodriguez (SEA - CF) 3', '4', '3.2', '0.4', '2.0', ' -1.0', '4', 'Mookie Betts (LAD - 2B,RF) 2', '5', '4.0', '1.1', '3.0', ' -1.0', '5', 'Corbin 

In [222]:
data = []

col_list = ['Rank', 'Player', '(Team, Position)', 'Best', 'Worst', 'Avg', 'Std Dev', 'ADP', 'vs. ADP', 'Notes']

# filter to only '(Team, Position)', 'Best', 'Notes' by item
cols = [item for item in col_list if item not in ['(Team, Position)', 'Best', 'Notes']]
print(cols)


data = [item for item in items_list if item not in col_list]

# seperate data into chunks of 7 for each row
data = [data[i:i+7] for i in range(0, len(data), 7)]

# create dataframe
df = pd.DataFrame(data, columns=cols)

print(df.head())

# split Player into Player_name and Other on the "("
df['Player_name'] = df['Player'].apply(lambda x: x.split('(')[0].strip())
df['Other'] = df['Player'].apply(lambda x: x.split('(')[1].replace(')', '').strip())
# remove Player column
df = df.drop('Player', axis=1)

# take the last character in Other and save it as "Best"
df['Best'] = df['Other'].apply(lambda x: x[-1])
# remove last character from Other
df['Other'] = df['Other'].apply(lambda x: x[:-1].strip())

# split Other into Team and Position on -
df['Team'] = df['Other'].apply(lambda x: x.split('-')[0].strip() if '-' in x else '')
df['Position'] = df['Other'].apply(lambda x: x.split('-')[1].strip() if '-' in x else '')


# trim Position colun to only everything before the first space
df['Position'] = df['Position'].apply(lambda x: x.split(' ')[0])

# remove Other column
df = df.drop('Other', axis=1)

# for  the Position column


# for the Position column, I want to duplicate the row for each position in the Position column (or for each , in Position)

# after that, I want to create listed_position which contains one of the multiple positions for that row
df['position_list'] = df['Position'].apply(lambda x: x.split(','))

print(df.head())

# duplicate rows for each position in position_list
df = df.explode('position_list')

# flip Position and position_list columns
position_list_to_add = df['Position']
df['Position'] = df['position_list']
df['position_list'] = position_list_to_add

# for the Position column, replace all instances of "LF", "RF" or "CF" with "OF" matching text exactly
df['Position'] = df['Position'].apply(lambda x: 'OF' if x in ['LF', 'RF', 'CF'] else x)

# sort by name
df = df.sort_values(by='Player_name')

['Rank', 'Player', 'Worst', 'Avg', 'Std Dev', 'ADP', 'vs. ADP']
  Rank                             Player Worst  Avg Std Dev  ADP vs. ADP
0    1      Ronald Acuna Jr. (ATL - RF) 1     1  1.0     0.0  1.0       ‐
1    2         Bobby Witt Jr. (KC - SS) 2     3  2.2     0.4  4.0    +2.0
2    3       Julio Rodriguez (SEA - CF) 3     4  3.2     0.4  2.0    -1.0
3    4       Mookie Betts (LAD - 2B,RF) 2     5  4.0     1.1  3.0    -1.0
4    5  Corbin Carroll (ARI - LF,CF,RF) 4     7  5.8     1.0  5.0       ‐
  Rank Worst  Avg Std Dev  ADP vs. ADP       Player_name Best Team  Position  \
0    1     1  1.0     0.0  1.0       ‐  Ronald Acuna Jr.    1  ATL        RF   
1    2     3  2.2     0.4  4.0    +2.0    Bobby Witt Jr.    2   KC        SS   
2    3     4  3.2     0.4  2.0    -1.0   Julio Rodriguez    3  SEA        CF   
3    4     5  4.0     1.1  3.0    -1.0      Mookie Betts    2  LAD     2B,RF   
4    5     7  5.8     1.0  5.0       ‐    Corbin Carroll    4  ARI  LF,CF,RF   

  position_

In [223]:
from espn_api.baseball import League  

# Create a league object
league = League(league_id=1893307610, year=2024, swid = "{2775F236-BFEF-4D4C-B5F2-36BFEF3D4C8E}", espn_s2= 'AECtTBz1STuXPdo4TMFTarhWjlgExxE7Vt7UFvOTscpbT6Ajd8qhO3EWuCuJ3xZrgGovAlw2Whkl1VPzDCGN7lpuRzzq7GtVhXtD2IvAELW%2FES61ZF3gMrGIzwjZ28dtDtATNFf%2B1UK5Sj%2FZWnq7e6aZY6nTfVb59UWRwkzXv6rq2l9bm8AT8xzYvdp0KN%2FbRjo4jqc2Sh%2FmSFVvj3lzMYMl1BeEqu5eE5aHZbCkPWzq1%2Fmf%2F%2BSQIXMbmZLzHq5zOuMUa2IWWxKNymv1o29nVcQA')




# {"swid":"{2775F236-BFEF-4D4C-B5F2-36BFEF3D4C8E}"}

# AECtTBz1STuXPdo4TMFTarhWjlgExxE7Vt7UFvOTscpbT6Ajd8qhO3EWuCuJ3xZrgGovAlw2Whkl1VPzDCGN7lpuRzzq7GtVhXtD2IvAELW%2FES61ZF3gMrGIzwjZ28dtDtATNFf%2B1UK5Sj%2FZWnq7e6aZY6nTfVb59UWRwkzXv6rq2l9bm8AT8xzYvdp0KN%2FbRjo4jqc2Sh%2FmSFVvj3lzMYMl1BeEqu5eE5aHZbCkPWzq1%2Fmf%2F%2BSQIXMbmZLzHq5zOuMUa2IWWxKNymv1o29nVcQA


#https://fantasy.espn.com/baseball/team?leagueId=1893307610&teamId=10

In [215]:
# Print the league name
league.free_agents(size = 10)

# grab my team
my_team = league.teams[8]

my_roster = my_team.roster

# 
my_roster_strings = [str(player).split('(')[1][:-1] for player in my_roster]

print(my_roster_strings)


['Bobby Witt Jr.', 'Fernando Tatis Jr.', 'Trea Turner', 'Austin Riley', 'Michael Harris II', 'Randy Arozarena', 'Adolis Garcia', 'Cody Bellinger', 'Mike Trout', 'Triston Casas', 'CJ Abrams', 'Andres Munoz', 'Paul Sewald', 'Andres Gimenez', 'Yainer Diaz', 'Jazz Chisholm Jr.', 'Zack Gelof', 'Jordan Walker', 'Shota Imanaga', 'Bryce Miller', 'TJ Friedl', 'Kyle Harrison', 'Eduardo Rodriguez', 'Thairo Estrada', 'Edwin Diaz', 'Luis Severino', 'Jared Jones', 'DL Hall', 'Michael Soroka']


In [216]:
# filter df["Player_name"] to only players in my_roster_strings
my_player_fantasypros = df[df["Player_name"].isin(my_roster_strings)]
print(my_player_fantasypros)

print(my_player_fantasypros["Position"])

    Rank Worst    Avg Std Dev    ADP vs. ADP         Player_name Best Team  \
40    41    69   44.6    12.6   38.0    -3.0       Adolis Garcia    5  TEX   
93    94   151   97.4    31.7  113.0   +19.0      Andres Gimenez    8  CLE   
88    89   118   93.0    13.6  107.0   +18.0        Andres Munoz    1  SEA   
17    18    20   19.0     1.1   18.0       ‐        Austin Riley    7  ATL   
1      2     3    2.2     0.4    4.0    +2.0      Bobby Witt Jr.    2   KC   
175  176   231  178.4    35.6  178.0    +2.0        Bryce Miller    9  SEA   
49    50    88   55.2    18.8   45.0    -5.0           CJ Abrams    6  WSH   
56    57    91   60.2    15.8   51.0    -6.0      Cody Bellinger    8  CHC   
56    57    91   60.2    15.8   51.0    -6.0      Cody Bellinger    8  CHC   
362  363   347  310.5    36.5  304.0   -59.0             DL Hall    4  MIL   
214  215   330  218.2    63.7  190.0   -25.0   Eduardo Rodriguez    3  ARI   
41    42    55   48.2     5.0   55.0   +13.0          Edwin Diaz

In [217]:
# grab all free agents
free_agents = league.free_agents(size = 10000)
print(free_agents)

free_agent_strings = [str(player).split('(')[1][:-1] for player in free_agents]
print(free_agent_strings)

# filter df["Player_name"] to only players in free_agent_strings
free_agent_fantasypros = df[df["Player_name"].isin(free_agent_strings)]

# sort by ascending Rank
free_agent_fantasypros = free_agent_fantasypros.sort_values(by = 'Rank')
print(free_agent_fantasypros.head())

[Player(Jose Alvarado), Player(Alex Lange), Player(Kyle Bradish), Player(Devin Williams), Player(Yennier Cano), Player(Ryan Pepiot), Player(Daulton Varsho), Player(Triston McKenzie), Player(Nestor Cortes), Player(Kutter Crawford), Player(Jon Gray), Player(Chas McCormick), Player(Ryan Pressly), Player(Bryan Woo), Player(Aaron Civale), Player(Anthony Rizzo), Player(Lars Nootbaar), Player(Gavin Williams), Player(Nathaniel Lowe), Player(Josiah Gray), Player(Brandon Pfaadt), Player(Vaughn Grissom), Player(Joel Payamps), Player(Andrew Abbott), Player(J.D. Martinez), Player(Robert Stephenson), Player(Alex Verdugo), Player(Jacob deGrom), Player(Jason Adam), Player(Bryan Abreu), Player(Gavin Stone), Player(Taylor Ward), Player(Colt Keith), Player(Hector Neris), Player(Andrew Heaney), Player(Jordan Hicks), Player(Andrew Vaughn), Player(Ryan McMahon), Player(Wilmer Flores), Player(Lucas Giolito), Player(A.J. Minter), Player(Jorge Polanco), Player(Kerry Carpenter), Player(Logan Allen), Player(Kevi

In [218]:
#group my_player_fantasypros by position and select max rank and Player_name
my_player_fantasypros_worst = my_player_fantasypros.groupby("Position").agg({'Rank': 'max', 'Player_name': 'first'}).reset_index()
# sort by position
my_player_fantasypros_worst = my_player_fantasypros_worst.sort_values(by = 'Position')
print(my_player_fantasypros_worst)

# group free_agent_fantasypros by position and select min rank and Player_name
free_agent_fantasypros_best = free_agent_fantasypros.groupby("Position").agg({'Rank': 'min', 'Player_name': 'first'}).reset_index()
# sort by position
free_agent_fantasypros_best = free_agent_fantasypros_best.sort_values(by = 'Position')
print(free_agent_fantasypros_best)


  Position Rank     Player_name
0       1B   72  Cody Bellinger
1       2B   94  Andres Gimenez
2       3B   18    Austin Riley
3        C  128     Yainer Diaz
4       DH  128     Yainer Diaz
5       OF   68   Adolis Garcia
6       RP   89    Andres Munoz
7       SP  286    Bryce Miller
8       SS   50  Bobby Witt Jr.
  Position Rank     Player_name
0           285      Tommy Pham
1       1B  189  Nathaniel Lowe
2       2B  134  Edouard Julien
3       3B  156       Max Muncy
4        C  243  Alejandro Kirk
5       DH  134  Edouard Julien
6       OF  125  Chas McCormick
7       RP  171   Jose Alvarado
8       SP  194    Aaron Civale
9       SS  229       Zach Neto
